In [ ]:
# Install wradlib first
# pip install wradlib

import wradlib as wrl
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append('../utils')

# Read RADOLAN file
# filename = '../data/dwd/recent/extracted/202501/raa01-sf_10000-2501131250-dwd---bin'
filename = '../data/dwd/extracted/202506/raa01-sf_10000-2506011250-dwd---bin'

# Read the RADOLAN composite
data, attrs = wrl.io.read_radolan_composite(filename)

print("Data shape:", data.shape)
print("Attributes:", attrs)
print("Data type:", data.dtype)
print("Data range:", data.min(), "to", data.max())

# Convert to mm/h (RADOLAN SF products are in mm/h * 100)
precip_data = data / 100.0

# Set no-data values to NaN
precip_data[data == -9999] = np.nan

In [ ]:
import wradlib.vis as vis

radar_coords = wrl.georef.get_radolan_grid(
    900, 900, wgs84=True)  # or wrl.georef.get_radolan_grid()
x, y = radar_coords[..., 0], radar_coords[..., 1]

fig, ax = plt.subplots(figsize=(10, 10))
pm = ax.pcolormesh(x, y, data, cmap="viridis", shading="auto")
plt.colorbar(pm, ax=ax, label="mm/h")
ax.set_title(f"RADOLAN {attrs['producttype']} - {attrs['datetime']}")
plt.axis('equal')
plt.show()

radar_coords.shape
radar_coords[0, 0]


# Radolan Coordinates
https://docs.wradlib.org/en/2.0.0/generated/wradlib.georef.rect.get_radolan_grid.html

- wrl.georef.get_radolan_grid(900,900) calculates x,y in DWD standard
- wrl.georef.get_radolan_grid(900,900, wsg84 = True) calculates wsg84 standard


In [ ]:
def crop_radolan_map(data, min_lat, max_lat, min_lon, max_lon):
    """
    Crop RADOLAN data to specified lat/lon bounds.
    
    Parameters:
    -----------
    data : np.array
        RADOLAN precipitation data (900x900)
    min_lat, max_lat : float
        Latitude bounds
    min_lon, max_lon : float  
        Longitude bounds
        
    Returns:
    --------
    cropped_data : np.array
        Cropped precipitation data
    cropped_coords : np.array
        Corresponding coordinate grid (cropped_rows, cropped_cols, 2)
    row_indices : tuple
        (min_row, max_row) indices used for cropping
    col_indices : tuple
        (min_col, max_col) indices used for cropping
    """
    # Get coordinate grid
    coords = wrl.georef.get_radolan_grid(900, 900, wsg84=True)
    lats = coords[..., 1]  # latitude is in index 1
    lons = coords[..., 0]  # longitude is in index 0

    # Find indices within bounds
    lat_mask = (lats >= min_lat) & (lats <= max_lat)
    lon_mask = (lons >= min_lon) & (lons <= max_lon)

    # Combined mask
    combined_mask = lat_mask & lon_mask

    # Get row and column ranges
    rows_with_data = np.any(combined_mask, axis=1)
    cols_with_data = np.any(combined_mask, axis=0)

    row_indices = np.where(rows_with_data)[0]
    col_indices = np.where(cols_with_data)[0]

    if len(row_indices) == 0 or len(col_indices) == 0:
        print("No data found within specified bounds")
        return None, None, None, None

    min_row, max_row = row_indices[0], row_indices[-1] + 1
    min_col, max_col = col_indices[0], col_indices[-1] + 1

    # Crop data and coordinates
    cropped_data = data[min_row:max_row, min_col:max_col]
    cropped_coords = coords[min_row:max_row, min_col:max_col]

    print(f"Original shape: {data.shape}")
    print(f"Cropped shape: {cropped_data.shape}")
    print(f"Row range: {min_row}-{max_row}, Col range: {min_col}-{max_col}")

    return cropped_data, cropped_coords, (min_row, max_row), (min_col, max_col)


# Example usage with Berlin bounds
berlin_min_lat, berlin_max_lat = 52, 52.7
berlin_min_lon, berlin_max_lon = 13.0, 13.8

cropped_data, cropped_coords, row_idx, col_idx = crop_radolan_map(
    precip_data, berlin_min_lat, berlin_max_lat, berlin_min_lon,
    berlin_max_lon)

if cropped_data is not None:
    # Visualize cropped data
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # Original data
    im1 = ax1.imshow(precip_data, cmap='viridis', aspect='auto')
    ax1.set_title("Original RADOLAN Data")
    ax1.add_patch(
        plt.Rectangle((col_idx[0], row_idx[0]),
                      col_idx[1] - col_idx[0],
                      row_idx[1] - row_idx[0],
                      fill=False,
                      edgecolor='red',
                      linewidth=2))
    plt.colorbar(im1, ax=ax1, label='mm/h')

    # Cropped data
    im2 = ax2.imshow(cropped_data, cmap='viridis', aspect='auto')
    ax2.set_title("Cropped Berlin Area")
    plt.colorbar(im2, ax=ax2, label='mm/h')

    plt.tight_layout()
    plt.show()

In [ ]:
# Add this cell to your notebook
import geopandas as gpd
import contextily as ctx
from shapely.geometry import Point
import matplotlib.pyplot as plt


# Convert DataFrame to GeoDataFrame
def create_precipitation_geodataframe(df):
    """Convert precipitation DataFrame to GeoDataFrame."""
    if len(df) == 0:
        return gpd.GeoDataFrame()

    # Create Point geometries from lat/lon
    geometry = [Point(lon, lat) for lon, lat in zip(df['lon'], df['lat'])]

    # Create GeoDataFrame with WGS84 projection (EPSG:4326)
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

    return gdf


# Create GeoDataFrame
precip_gdf = create_precipitation_geodataframe(df)
print(f"Created GeoDataFrame with {len(precip_gdf)} precipitation points"
      )  # Enhanced visualization with map overlay
fig, ax = plt.subplots(1, 1, figsize=(15, 12))

if len(precip_gdf) > 0:
    # Convert to Web Mercator for basemap overlay
    precip_gdf_mercator = precip_gdf.to_crs('EPSG:3857')

    # Plot precipitation data
    scatter = precip_gdf_mercator.plot(column='precipitation_mm_h',
                                       ax=ax,
                                       cmap='Blues',
                                       markersize=15,
                                       alpha=0.7,
                                       legend=True,
                                       legend_kwds={
                                           'label': 'Precipitation (mm/h)',
                                           'shrink': 0.8
                                       })

    # Add basemap (OpenStreetMap)
    try:
        ctx.add_basemap(ax,
                        crs=precip_gdf_mercator.crs.to_string(),
                        source=ctx.providers.OpenStreetMap.Mapnik,
                        alpha=0.6)
    except:
        print("Could not add basemap - continuing without it")

    # Set title and labels
    ax.set_title(
        'RADOLAN Precipitation Data - Berlin Area\nwith OpenStreetMap Overlay',
        fontsize=14,
        pad=20)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    # Remove axis ticks for cleaner look
    ax.set_xticks([])
    ax.set_yticks([])

else:
    ax.text(0.5,
            0.5,
            "No precipitation data in Berlin area for this timestamp",
            ha='center',
            va='center',
            transform=ax.transAxes)

plt.tight_layout()
plt.show()

# Visualizing cropping

In [ ]:
import sys

sys.path.append("..")

import importlib
from src.utils import data_processing_dwd

importlib.reload(data_processing_dwd)

import wradlib as wrl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Read RADOLAN file

filename = '../data/dwd/extracted/202501/raa01-sf_10000-2501301250-dwd---bin'

# Read the RADOLAN composite
data, attrs = wrl.io.read_radolan_composite(filename)

print("Data shape:", data.shape)
print("Attributes:", attrs)
print("Data type:", data.dtype)
print("Data range:", data.min(), "to", data.max())

# Set no-data values to NaN
data[data == -9999] = np.nan

import wradlib.vis as vis

radar_coords = wrl.georef.get_radolan_grid(900, 900, wgs84=True)
x, y = radar_coords[..., 0], radar_coords[..., 1]

fig, ax = plt.subplots(figsize=(10, 10))
pm = ax.pcolormesh(x, y, data, cmap="viridis", shading="auto")
# Define Berlin bounds
berlin_min_lat, berlin_max_lat = 52.29, 52.9
berlin_min_lon, berlin_max_lon = 13.11, 13.79

# Add red rectangle outline for Berlin area
rect = patches.Rectangle(
    (berlin_min_lon, berlin_min_lat),  # Bottom-left corner (lon, lat)
    berlin_max_lon - berlin_min_lon,  # Width (longitude difference)
    berlin_max_lat - berlin_min_lat,  # Height (latitude difference)
    linewidth=2,
    edgecolor='red',
    facecolor='none',  # Transparent fill
    alpha=0.8)
ax.add_patch(rect)
plt.colorbar(pm, ax=ax, label="mm/h")
ax.set_title(f"RADOLAN {attrs['producttype']} - {attrs['datetime']}")
plt.axis('equal')
plt.show()


In [ ]:
# Example usage with Berlin bounds

berlin_min_lat, berlin_max_lat = 52.29, 52.9
berlin_min_lon, berlin_max_lon = 13.11, 13.79

cropped_data, cropped_coords = data_processing_dwd.crop_radolan_map(
    data, berlin_min_lat, berlin_max_lat, berlin_min_lon, berlin_max_lon)

fig, ax = plt.subplots(figsize=(8, 6))
pm = ax.pcolormesh(
    cropped_coords[..., 0],  # longitude
    cropped_coords[..., 1],  # latitude
    cropped_data,
    cmap="viridis",
    shading="auto")
plt.colorbar(pm, ax=ax, label="mm/h")
ax.set_title("Cropped RADOLAN Data (Berlin Area)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()